In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

## Import framework main libs

In [ ]:
from tmqrfeed.manager import DataManager
from tmqrstrategy.optimizers import OptimizerBase, OptimizerGenetic


## Import strategy from module

In [ ]:
from tmqralphas.strategy_dsp_lpbp_combination import Strategy_DSP_LPBP_Combination

# Strategy inheritance trick!

We inherited Strategy_DSP_LPBP_Combination strategy the logic remains the same, but position management in different.

In [ ]:
import pandas as pd
from tmqrfeed.quotes import QuoteIndex
from tmqr.logs import log
from tmqr.errors import PositionNotFoundError


class Strategy_DSP_LPBP_HedgedByIndex(Strategy_DSP_LPBP_Combination):
    def setup(self):
        # Call parent Strategy_DSP_LPBP_Combination.setup() -> tmqrstrategy.strategy_alpha.StrategyAlpha.setup()
        super().setup()
        
        #
        # We have to add index we wanted to hedge by
        #
        HEDGE_IDX_NAME = self.context['index_hedge_name']        
        self.dm.series_extra_set('index_hedge', QuoteIndex, HEDGE_IDX_NAME, set_session=True, check_session=True)
    
    #
    # This is exact copy/paste of souce code of tmqrstrategy.strategy_alpha.StrategyAlpha.calculate_position() method
    #
    def calculate_position(self, date: datetime, exposure_record: pd.DataFrame):
        """
        This alpha just replicates EXO/SmartEXO index position


        This method used for position construction based on exposure information returned from calculate(),
        here you can initiate (replicate) EXO index position or setup any custom position you want.
        """
        # Get the position of Quote algo (in this case current cont futures)
        primary_quotes_position = self.dm.position()

        # ALSO you can get secondary positions
        # secondary_position = self.dm.position('CONTFUT')

        # get net exposure for all members
        # exposure_record - is a slice of exposures results of picked alphas at 'date'

        # We are calling sum() because we have multiple records of 'exposure'
        # 1-alpha member of best in the swarm per row
        if 'exposure' not in exposure_record:
            raise StrategyError(
                "'exposure_record' expected to have 'exposure' column, check alpha's calculate(...) method "
                "to make sure that it returns valid pandas.DataFrame with exposure column or just check "
                "for 'return self.exposure(...)' in the last line")
        exposure = exposure_record['exposure'].sum()

        #
        # Just replicate primary quotes position
        #
        replicated_pos = primary_quotes_position.get_net_position(date)
        self.position.add_net_position(date, replicated_pos, qty=exposure)
        
        
        #
        # Do new position management magic here
        #
        index_hedge_position = self.dm.position('index_hedge')
        try:
            hedge_position_rec = index_hedge_position.get_net_position(date)
            # Add index position as hedge
            # NOTE: exposure - is a alpha's exporure of trade, when alpha is out of market
            #                  exposure equals 0, this means that means no position and hedge
            # NOTE: self.context['index_hedge_direction'] allowed 1, -1, or even 0 - i.e. no hedge
            self.position.add_net_position(date, hedge_position_rec, qty=abs(exposure)*self.context['index_hedge_direction'])
        except PositionNotFoundError as exc:
            log.error(f"Couldn't find hedged index position! {exc}")
            
        #print(f'Exposure: {exposure}')
        #print(self.position)
        
    

In [ ]:
ALPHA_CONTEXT = {
        'name': 'ES_NewStrategy_DSP_LPBP_Combination_With_IndexHedge', # Global alpha name, which be used for load/save from DB
        'context': { # Strategy specific settings
            # These settings only applycable for alphas derived from StrategyAlpha strategy 
            # StrategyAlpha - is a classic EXO/SmartEXO based alpha
            'index_name': 'US.ES_ContFutEOD',      # Name of EXO index to trade
            
            # !!! NEW RECORD
            'index_hedge_name': 'US.ES_EXOSpreadFixed', # Name of the index used for hedge
            'index_hedge_direction':0, # ALLOWED 1, -1, or even 0 - i.e. no hedge
            #
            
            'costs_per_option': 3.0,
            'costs_per_contract': 3.0,
        },
        'wfo_params': {
            'window_type': 'rolling',  # Rolling window for IIS values: rolling or expanding
            'period': 'M',  # Period of rolling window 'M' - monthly or 'W' - weekly
            'oos_periods': 3,  # Number of months is OOS period
            'iis_periods': 12,
            # Number of months in IIS rolling window (only applicable for 'window_type' == 'rolling')
        },
        'wfo_optimizer_class': OptimizerBase, 
        'wfo_optimizer_class_kwargs': {
            'nbest_count': 1,
            'nbest_fitness_method': 'max'
        },
          'wfo_opt_params': [
                ('Direction', [1]),
                ('LP order',[1,2,3,4,5,6,7,8,9]),
                ('LP freq',[0.5982, 0.5924, 0.5937, 0.5938, 0.5908]), # 0 > f < 1
                ('BP order',[0,1,2,3,4,5,6]),
                ('BP start freq', [0.2986, 0.296, 0.297, 0.2944, 0.2995]),
                ('BP stop freq',   [0.2514, 0.2548, 0.2535, 0.2569, 0.2509]),
                ('BP multiplier',[-4, -3] ),
                ('Rule index',   [2] ),                        
            ],
        'wfo_members_count': 1,
        'wfo_costs_per_contract': 0.0,
        'wfo_scoring_type': 'netprofit'
    }

# Run the alpha

In [ ]:
# DataManager is a core class of the framework
dm = DataManager()

# Init alpha class and run
alpha = Strategy_DSP_LPBP_HedgedByIndex(dm, **ALPHA_CONTEXT)    

In [ ]:
alpha.run()

# Equity

In [ ]:
alpha.stats['series']['equity'].plot()

# Position holdings

#### Position hedge proof: alpha uses ES furues as primary position and options (from index position) as hedge

In [ ]:
for asset, pos_rec in alpha.position.get_net_position(pd.Timestamp('2017-05-02 12:45:00-07:00')).items():
    print(f'{asset} qty: {pos_rec[2]}')